In [1]:
import os
from icecube import icetray, dataio, dataclasses, simclasses
from collections import Counter

In [2]:
icetray.I3Logger.global_logger = icetray.I3NullLogger()

In [3]:
raw_file_example = "/project/6008051/pone_simulation/MC10-000002-nu_mu-2_7-LeptonInjector-PROPOSAL-clsim/Photon/cls_4910.i3"

In [4]:
print("exists:", os.path.exists(raw_file_example))
print("size (MB):", os.path.getsize(raw_file_example)/1e6 if os.path.exists(raw_file_example) else None)

exists: True
size (MB): 2836.732093


In [5]:
f = dataio.I3File(raw_file_example, "r")

In [6]:
f

In [7]:
print(f)

In [8]:
counts = Counter()
total = 0

while f.more():
    frame = f.pop_frame()
    total += 1
    counts[str(frame.Stop)] += 1

print("total frames:", total)
print("by Stop:")
for k, v in counts.most_common():
    print(f"  {k}: {v}")


total frames: 201
by Stop:
  DAQ: 200
  TrayInfo: 1


In [9]:
f = dataio.I3File(raw_file_example, "r")

i = 0
tray_indices = []

while f.more():
    frame = f.pop_frame()
    if frame.Stop == icetray.I3Frame.TrayInfo:
        tray_indices.append(i)
    i += 1

print("TrayInfo indices (0-based):", tray_indices)

TrayInfo indices (0-based): [0]


In [10]:
f = dataio.I3File(raw_file_example, "r")
frame0 = f.pop_frame()  # TrayInfo

for k in frame0.keys():
    v = frame0[k]
    print("KEY:", k)
    print("  type:", type(v))
    print("  value:", v)

KEY: 2025-10-02T15:01:23.012653
  type: <class 'icecube._icetray.I3TrayInfo'>
  value: ============================= configuration ==============================
      Icetray run started:  Thu Oct  2 08:01:23 2025
                  on host:  fc11017 (Linux-5.14.0-427.37.1.el9_4.x86_64/x86_64/gcc-11.4.0)
                  as user:  cmiller
          and gcc version:  11.4.0
                  svn url:  
             svn revision:  0

===========================   svn:externals   ============================

=============================   services    ==============================
=============================    modules    ==============================
reader (I3Reader)
  DropBuffers
    Description :  Tell I3Frames not to cache buffers of serialized frameobject data (this saves memory at the expense of processing speed and the ability to passthru unknown frame objects)
    Default     :  False
    Configured  :  None

  Filename
    Description :  Filename to read.  Use either this 

The first frame is a **TrayInfo** frame, which stores an `I3TrayInfo` object describing how the file was produced (the IceTray “recipe”). It lists the modules that were run (e.g., injector/propagator/clsim), their configuration parameters, and which frame streams were written (e.g., TrayInfo and DAQ). It’s metadata about the processing pipeline, not the event data itself.


In [11]:
f = dataio.I3File(raw_file_example, "r")

daq = None
while f.more():
    frame = f.pop_frame()
    if frame.Stop == icetray.I3Frame.DAQ:
        daq = frame
        break

print("got DAQ:", daq is not None)
print("n_keys:", len(daq.keys()) if daq else None)


got DAQ: True
n_keys: 7


In [12]:
keys = list(daq.keys())
print(keys)

['MMCTrackList', 'I3Photons', 'I3MCTree_postprop', 'I3MCTree_RNGState', 'I3MCTree', 'I3EventHeader', 'EventProperties']


In [13]:
print(daq)

[ I3Frame  (DAQ):
  'EventProperties' [DAQ] ==> LeptonInjector::BasicEventProperties (140)
  'I3EventHeader' [DAQ] ==> I3EventHeader (99)
  'I3MCTree' [DAQ] ==> TreeBase::Tree<I3Particle, I3ParticleID, i3hash<I3ParticleID> > (422)
  'I3MCTree_RNGState' [DAQ] ==> I3GSLRandomServiceState (87)
  'I3MCTree_postprop' [DAQ] ==> TreeBase::Tree<I3Particle, I3ParticleID, i3hash<I3ParticleID> > (422)
  'I3Photons' [DAQ] ==> I3Map<ModuleKey, I3Vector<I3CompressedPhoton> > (53)
  'MMCTrackList' [DAQ] ==> I3Vector<I3MMCTrack> (40)
]



## Only in the DAQ frame (raw data)
- `I3Photons`

## Only in the Physics frame (POM Response output)
- `EventPulseSeries`
- `Noise_Dark`
- `Noise_K40`
- `PMT_Response`
- `PMT_Response_nonoise`
- `TriggerTime_3PMT_1DOM`

## Present in both (DAQ + Physics)
- `EventProperties`
- `I3EventHeader`
- `I3MCTree`
- `I3MCTree_RNGState`
- `I3MCTree_postprop`
- `MMCTrackList`


In [14]:
photons = daq["I3Photons"]

print("type:", type(photons))
print("n ModuleKeys:", len(photons))


type: <class 'icecube._simclasses.I3CompressedPhotonSeriesMap'>
n ModuleKeys: 0


In [15]:
print(photons)

[]


in the first DAQ frame, "I3Photons" is empty.

In [16]:
f = dataio.I3File(raw_file_example, "r")

n_daq = 0
n_nonempty = 0
nonempty_indices = []  # DAQ index (0-based, sadece DAQ'lar sayılıyor)

daq_idx = -1
while f.more():
    frame = f.pop_frame()
    if frame.Stop != icetray.I3Frame.DAQ:
        continue

    n_daq += 1
    daq_idx += 1

    photons = frame["I3Photons"]
    if len(photons) > 0:
        n_nonempty += 1
        nonempty_indices.append(daq_idx)

print("total DAQ frames:", n_daq)
print("DAQ frames with non-empty I3Photons:", n_nonempty)
print("non-empty DAQ indices:", nonempty_indices)


total DAQ frames: 200
DAQ frames with non-empty I3Photons: 57
non-empty DAQ indices: [6, 8, 9, 11, 17, 19, 26, 28, 29, 34, 43, 44, 47, 48, 51, 57, 60, 63, 70, 80, 82, 83, 84, 88, 90, 105, 109, 111, 119, 120, 124, 125, 126, 127, 129, 130, 133, 134, 135, 137, 140, 146, 147, 149, 158, 162, 164, 166, 167, 168, 178, 179, 180, 190, 191, 193, 199]


In [17]:
f = dataio.I3File(raw_file_example, "r")

frame = None
for i in range(8):  # 0..7
    frame = f.pop_frame()

print("file-frame index 6 Stop:", frame.Stop)
print("keys:", list(frame.keys()))


file-frame index 6 Stop: DAQ
keys: ['MMCTrackList', 'I3Photons', 'I3MCTree_postprop', 'I3MCTree_RNGState', 'I3MCTree', 'I3EventHeader', 'EventProperties']


In [18]:
# print(frame['I3Photons'])

In [19]:
frame['I3Photons'][dataclasses.ModuleKey(231, 18)]

In [20]:
print(frame['I3Photons'][dataclasses.ModuleKey(231, 18)])

[[I3CompressedPhoton: 
        Time: 968.141
      Weight: 1
  Wavelength: 4.50997e-07
      Zenith: 2.09724
     Azimuth: 1.79005
    Position: (-0.0376072,0.198429,0.0760731)
  Group Vel.: 0.21702
  ParticleID: 10320399505715655872,133)]]


In [21]:
len(frame['I3Photons'][dataclasses.ModuleKey(231, 18)])

1

In [22]:
print(frame['I3Photons'][dataclasses.ModuleKey(247, 16)])

[[I3CompressedPhoton: 
        Time: 488.207
      Weight: 1
  Wavelength: 4.41561e-07
      Zenith: 1.60039
     Azimuth: 1.05431
    Position: (0.0523264,0.205239,-0.0417951)
  Group Vel.: 0.216693
  ParticleID: 10320399505715655872,133)], [I3CompressedPhoton: 
        Time: 553.593
      Weight: 1
  Wavelength: 4.70742e-07
      Zenith: 1.1452
     Azimuth: 1.26438
    Position: (0.0464862,0.113896,0.177392)
  Group Vel.: 0.217627
  ParticleID: 10320399505715655872,133)], [I3CompressedPhoton: 
        Time: 489.622
      Weight: 1
  Wavelength: 4.47005e-07
      Zenith: 1.58093
     Azimuth: 1.08256
    Position: (-0.00426921,0.138696,-0.165342)
  Group Vel.: 0.216885
  ParticleID: 10320399505715655872,133)], [I3CompressedPhoton: 
        Time: 503.867
      Weight: 1
  Wavelength: 4.46369e-07
      Zenith: 1.56747
     Azimuth: 1.10571
    Position: (0.0123699,0.182144,0.114782)
  Group Vel.: 0.216863
  ParticleID: 10320399505715655872,133)], [I3CompressedPhoton: 
        Time: 513

In [23]:
len(frame['I3Photons'][dataclasses.ModuleKey(247, 16)])

5

In [24]:
# ModuleKey neydi, string ve OM miydi

From the Confluence page, the DOM radius is listed as **DOMRadius = 0.21590 m**.

To sanity-check what the `Position (x, y, z)` field in `I3CompressedPhoton`, I computed $ r = \sqrt{x^2 + y^2 + z^2} $ for three example photons (from the outputs of the cells 22, 22 above). The resulting radii were:

- 0.2158135  
- 0.2158887  
- 0.2158731  

These values are all very close to 0.21590 m, which strongly suggests that the `(x, y, z)` coordinates are given in the DOM’s local coordinate system (origin near the DOM center) and that `Position` corresponds to the point where the photon intersects/enters the DOM surface.


In [25]:
# neden bazi I3Photonlar bos?

In [26]:
mk = dataclasses.ModuleKey(247, 16)
ts = [p.time for p in frame["I3Photons"][mk]]  # p is I3CompressedPhoton
ts

[488.2065734863281,
 553.592529296875,
 489.6219482421875,
 503.8673095703125,
 513.5278930664062]

In [27]:
hdr = frame["I3EventHeader"]
hdr.start_time if hasattr(hdr, "start_time") else hdr

I3Time(   0,                 0L)

In [28]:
mctree = frame["I3MCTree_postprop"]  # veya "I3MCTree"



In [29]:
print(mctree.primaries)

[[ I3Particle MajorID : 10320399505715655872
             MinorID : 136
              Zenith : 2.41197
             Azimuth : 5.61741
                   X : -589.976
                   Y : 399.972
                   Z : 296.257
                Time : 0
              Energy : 993.477
               Speed : 0.299792
              Length : 0
                Type : NuMu
        PDG encoding : 14
               Shape : Primary
              Status : NotSet
            Location : Anywhere
]]


In [30]:
prim = mctree.primaries[0]


In [31]:
prim.time, prim.pos

(0.0, I3Position(-589.976,399.972,296.257))

In [32]:
type(frame["I3Photons"])

icecube._simclasses.I3CompressedPhotonSeriesMap

In [33]:
photons_map = frame["I3Photons"]  # I3CompressedPhotonSeriesMap

# map'ten bir key al (ilk ModuleKey)
mk = next(iter(photons_map.keys()))
mk

ModuleKey(231,18)

In [34]:

# o DOM/module için photon listesi
series = photons_map[mk]


In [35]:

print("map type:", type(photons_map))
print("key type:", type(mk))
print("series type:", type(series))
print("n photons in series:", len(series))

if len(series) > 0:
    print("first photon type:", type(series[0]))


map type: <class 'icecube._simclasses.I3CompressedPhotonSeriesMap'>
key type: <class 'icecube._dataclasses.ModuleKey'>
series type: <class 'icecube._simclasses.I3CompressedPhotonSeries'>
n photons in series: 1
first photon type: <class 'icecube._simclasses.I3CompressedPhoton'>


In [36]:
mk = dataclasses.ModuleKey(231, 18)


In [37]:
print(frame['I3Photons'][dataclasses.ModuleKey(231, 18)][0])

[I3CompressedPhoton: 
        Time: 968.141
      Weight: 1
  Wavelength: 4.50997e-07
      Zenith: 2.09724
     Azimuth: 1.79005
    Position: (-0.0376072,0.198429,0.0760731)
  Group Vel.: 0.21702
  ParticleID: 10320399505715655872,133)]


In [38]:
print(type(frame['I3Photons'][dataclasses.ModuleKey(231, 18)][0]))

<class 'icecube._simclasses.I3CompressedPhoton'>


In [39]:
print(type(frame['I3Photons'][dataclasses.ModuleKey(231, 18)]))

<class 'icecube._simclasses.I3CompressedPhotonSeries'>


In [40]:
########

In [41]:
mctree = frame["I3MCTree"]
prim = list(mctree.primaries)[0]
print("Primary time:", prim.time, "ns")
print("Primary pos :", prim.pos)


Primary time: 0.0 ns
Primary pos : I3Position(-589.976,399.972,296.257)


In [42]:
from icecube import dataclasses

key = dataclasses.ModuleKey(231, 18)
ph = frame["I3Photons"][key][0]


In [43]:
print(ph)

[I3CompressedPhoton: 
        Time: 968.141
      Weight: 1
  Wavelength: 4.50997e-07
      Zenith: 2.09724
     Azimuth: 1.79005
    Position: (-0.0376072,0.198429,0.0760731)
  Group Vel.: 0.21702
  ParticleID: 10320399505715655872,133)]


In [44]:
ph.particleMajorID

10320399505715655872

In [45]:
ph.particleMinorID

133

In [46]:
from icecube import dataclasses

key = dataclasses.ModuleKey(231,18)
ph = frame["I3Photons"][key][0]
maj, minr = ph.particleMajorID, ph.particleMinorID

def find_in_tree(tree):
    for p in tree:
        if p.id.majorID == maj and p.id.minorID == minr:
            return p
    return None

p1 = find_in_tree(frame["I3MCTree"])
p2 = find_in_tree(frame["I3MCTree_postprop"])

print("In I3MCTree        :", p1 is not None)
print("In I3MCTree_PostProp:", p2 is not None)
print("p1:", p1)
print("p2:", p2)


In I3MCTree        : True
In I3MCTree_PostProp: True
p1: [ I3Particle MajorID : 10320399505715655872
             MinorID : 133
              Zenith : 2.4186
             Azimuth : 5.63293
                   X : -589.976
                   Y : 399.972
                   Z : 296.257
                Time : 0
              Energy : 814.357
               Speed : 0.299792
              Length : nan
                Type : Hadrons
        PDG encoding : -2000001006
               Shape : Cascade
              Status : NotSet
            Location : InIce
]
p2: [ I3Particle MajorID : 10320399505715655872
             MinorID : 133
              Zenith : 2.4186
             Azimuth : 5.63293
                   X : -589.976
                   Y : 399.972
                   Z : 296.257
                Time : 0
              Energy : 814.357
               Speed : 0.299792
              Length : nan
                Type : Hadrons
        PDG encoding : -2000001006
               Shape : Cascade
  

In [47]:
from icecube import dataio, simclasses
from collections import Counter

def photon_map_keys(frame):
    """Frame'deki tüm I3CompressedPhotonSeriesMap key'lerini döndür."""
    keys = []
    for k in frame.keys():
        try:
            v = frame[k]
        except Exception:
            continue
        if isinstance(v, simclasses.I3CompressedPhotonSeriesMap):
            keys.append(k)
    return keys

def pick_tree(frame, prefer_postprop=True):
    """
    Photon ParticleID'lerinin refer ettiği en olası MCTree'yi seç.
    Bu dosyada PostProp key'i: 'I3MCTree_postprop'
    """
    candidates = []
    if prefer_postprop:
        candidates.append("I3MCTree_postprop")
    candidates.append("I3MCTree")

    for k in candidates:
        if k in frame:
            return frame[k], k
    return None, None

def src_class(p):
    """Kaynak parçacığı kaba sınıflandır."""
    if p is None:
        return "UNKNOWN"

    t = str(p.type)
    s = str(p.shape)

    if "Mu" in t:
        return "MUON"

    # muon propagation boyunca doğan secondary/loss parçacıklar çoğunlukla time>0 olur
    if getattr(p, "time", 0) > 0:
        return "SRC_TIME>0"

    if "Track" in s:
        return "TRACK"
    if "Cascade" in s or "Hadrons" in t:
        return "CASCADE"

    return "OTHER"

def scan_frame(frame, prefer_postprop=True):
    """Tek bir frame'de photonları tarar; photon map yoksa None döner."""
    pkeys = photon_map_keys(frame)
    if not pkeys:
        return None

    tree, treekey = pick_tree(frame, prefer_postprop=prefer_postprop)
    if tree is None:
        return None

    # hızlı lookup
    lut = {(p.id.majorID, p.id.minorID): p for p in tree}

    total = 0
    muon_ph = 0
    timegt0_ph = 0
    counts = Counter()

    for pk in pkeys:
        photmap = frame[pk]
        for omkey, series in photmap.items():
            for ph in series:
                total += 1
                p = lut.get((ph.particleMajorID, ph.particleMinorID), None)

                cls = src_class(p)
                counts[cls] += 1

                if cls == "MUON":
                    muon_ph += 1
                if p is not None and getattr(p, "time", 0) > 0:
                    timegt0_ph += 1

    hdr = frame["I3EventHeader"] if "I3EventHeader" in frame else None
    tag = {
        "stop": str(frame.Stop),
        "run": getattr(hdr, "run_id", None),
        "event": getattr(hdr, "event_id", None),
        "subevent": getattr(hdr, "sub_event_id", None),
    }

    return {
        "tag": tag,
        "tree": treekey,
        "photon_keys": pkeys,
        "total": total,
        "muon_ph": muon_ph,
        "timegt0_ph": timegt0_ph,
        "counts": counts,
    }

def scan_file(path, max_frames=None, prefer_postprop=True):
    """
    Dosyadaki tüm frame'leri (DAQ dahil) gezer, photon map olanları toplar.
    """
    f = dataio.I3File(path)

    agg = Counter()
    frames_with_photons = []
    n = 0

    while f.more():
        fr = f.pop_frame()
        n += 1
        if max_frames is not None and n > max_frames:
            break

        res = scan_frame(fr, prefer_postprop=prefer_postprop)
        if res is None:
            continue

        agg.update(res["counts"])
        frames_with_photons.append(res)

    print("Frames scanned:", n)
    print("Frames with photon maps:", len(frames_with_photons))
    print("Aggregate photon source classes:", dict(agg))

    mu_frames = [r for r in frames_with_photons if r["muon_ph"] > 0]
    t0_frames = [r for r in frames_with_photons if r["timegt0_ph"] > 0]

    print("Frames with MUON-tagged photons:", len(mu_frames))
    print("Frames with any src.time>0 photons:", len(t0_frames))

    mu_frames.sort(key=lambda r: r["muon_ph"], reverse=True)
    print("\nTop frames by MUON photons:")
    for r in mu_frames[:10]:
        print(r["tag"], "muon_ph=", r["muon_ph"], "total=", r["total"],
              "tree=", r["tree"], "pkeys=", r["photon_keys"])

    t0_frames.sort(key=lambda r: r["timegt0_ph"], reverse=True)
    print("\nTop frames by src.time>0 photons (muon-prop/loss candidate):")
    for r in t0_frames[:10]:
        print(r["tag"], "time>0_ph=", r["timegt0_ph"], "total=", r["total"],
              "tree=", r["tree"], "pkeys=", r["photon_keys"])

    return frames_with_photons, agg


In [48]:
frames_with_photons, agg = scan_file(raw_file_example, max_frames=None, prefer_postprop=True)


Frames scanned: 201
Frames with photon maps: 200
Aggregate photon source classes: {'CASCADE': 45217890, 'MUON': 173748, 'SRC_TIME>0': 5230261}
Frames with MUON-tagged photons: 50
Frames with any src.time>0 photons: 52

Top frames by MUON photons:
{'stop': 'DAQ', 'run': 4910, 'event': 64, 'subevent': 0} muon_ph= 20701 total= 36841 tree= I3MCTree_postprop pkeys= ['I3Photons']
{'stop': 'DAQ', 'run': 4910, 'event': 35, 'subevent': 0} muon_ph= 12629 total= 380890 tree= I3MCTree_postprop pkeys= ['I3Photons']
{'stop': 'DAQ', 'run': 4910, 'event': 168, 'subevent': 0} muon_ph= 12440 total= 131538 tree= I3MCTree_postprop pkeys= ['I3Photons']
{'stop': 'DAQ', 'run': 4910, 'event': 120, 'subevent': 0} muon_ph= 11663 total= 12273 tree= I3MCTree_postprop pkeys= ['I3Photons']
{'stop': 'DAQ', 'run': 4910, 'event': 20, 'subevent': 0} muon_ph= 11312 total= 24183 tree= I3MCTree_postprop pkeys= ['I3Photons']
{'stop': 'DAQ', 'run': 4910, 'event': 141, 'subevent': 0} muon_ph= 9193 total= 9348 tree= I3MCTree_

In [51]:
import pandas as pd
from icecube import dataio, simclasses

def _find_photon_map_keys(frame):
    keys = []
    for k in frame.keys():
        try:
            v = frame[k]
        except Exception:
            continue
        if isinstance(v, simclasses.I3CompressedPhotonSeriesMap):
            keys.append(k)
    return keys

def _find_iterable_mctree_keys(frame):
    """
    Frame'de 'MCTree' geçen ama iterable olmayan (örn RNGState) objeleri dışarıda bırakır.
    """
    out = []
    for k in frame.keys():
        if "MCTree" not in k:
            continue
        try:
            obj = frame[k]
        except Exception:
            continue
        try:
            iter(obj)  # iterable mı?
            out.append(k)
        except TypeError:
            continue
    return out

def _build_pid_lookup_for_trees(frame, tree_keys):
    """
    tree_key -> {(major, minor): I3Particle}
    """
    lut_by_tree = {}
    for tk in tree_keys:
        tree = frame[tk]
        lut = {}
        for p in tree:
            lut[(int(p.id.majorID), int(p.id.minorID))] = p
        lut_by_tree[tk] = lut
    return lut_by_tree

def photons_to_dataframe(
    path: str,
    *,
    only_daq: bool = True,
    max_frames: int | None = None,
    max_daq_frames: int | None = None,
    max_rows: int | None = 200_000,
):
    f = dataio.I3File(path)

    rows = []
    file_frame_index = 0
    daq_index = 0

    while f.more():
        fr = f.pop_frame()
        file_frame_index += 1

        stop_name = str(fr.Stop)
        if only_daq and stop_name != "DAQ":
            continue

        if stop_name == "DAQ":
            daq_index += 1
            if max_daq_frames is not None and daq_index > max_daq_frames:
                break

        if max_frames is not None and file_frame_index > max_frames:
            break

        photon_keys = _find_photon_map_keys(fr)
        if not photon_keys:
            continue

        tree_keys = _find_iterable_mctree_keys(fr)
        lut_by_tree = _build_pid_lookup_for_trees(fr, tree_keys)

        hdr = fr["I3EventHeader"] if "I3EventHeader" in fr else None
        run_id = getattr(hdr, "run_id", None)
        event_id = getattr(hdr, "event_id", None)
        subevent_id = getattr(hdr, "sub_event_id", None)

        for pk in photon_keys:
            photmap = fr[pk]

            for omkey, series in photmap.items():
                # ModuleKey parse
                try:
                    s = int(omkey.string)
                    om = int(omkey.om)
                except Exception:
                    s = None
                    om = None

                for i_ph, ph in enumerate(series):
                    pid = (int(ph.particleMajorID), int(ph.particleMinorID))

                    found_trees = []
                    particle_str_by_tree = {}
                    sig_parts = []

                    for tk in tree_keys:
                        lut = lut_by_tree[tk]          # <-- FIX: tree’ye ait lut
                        if pid in lut:
                            found_trees.append(tk)
                            particle_str_by_tree[tk] = str(lut[pid])  # <-- FIX
                            sig_parts.append(f"{tk}:1")
                        else:
                            sig_parts.append(f"{tk}:0")

                    presence_signature = "|".join(sig_parts) if sig_parts else ""
                    if len(found_trees) == 0:
                        presence_case = "none"
                    elif len(found_trees) == 1:
                        presence_case = f"only:{found_trees[0]}"
                    else:
                        presence_case = "multiple"

                    # photon position (DOM-local olabilir)
                    try:
                        posx, posy, posz = float(ph.pos.x), float(ph.pos.y), float(ph.pos.z)
                    except Exception:
                        posx = posy = posz = None

                    rows.append({
                        "file_frame_index": file_frame_index,
                        "daq_index": daq_index if stop_name == "DAQ" else None,
                        "stop": stop_name,
                        "run_id": run_id,
                        "event_id": event_id,
                        "sub_event_id": subevent_id,

                        "photon_map_key": pk,

                        "modulekey": str(omkey),
                        "string": s,
                        "om": om,
                        "photon_index": i_ph,

                        "photon_time": float(ph.time),
                        "photon_weight": float(ph.weight),
                        "photon_wavelength": float(ph.wavelength),
                        "photon_zenith": float(ph.zenith),
                        "photon_azimuth": float(ph.azimuth),
                        "photon_group_velocity": float(ph.groupVelocity),
                        "photon_pos_x": posx,
                        "photon_pos_y": posy,
                        "photon_pos_z": posz,

                        "pid_major": pid[0],
                        "pid_minor": pid[1],

                        "mctree_keys_in_frame": list(tree_keys),
                        "pid_found_in_trees": list(found_trees),
                        "pid_presence_signature": presence_signature,
                        "pid_presence_case": presence_case,
                        "particle_str_by_tree": particle_str_by_tree,
                    })

                    if max_rows is not None and len(rows) >= max_rows:
                        return pd.DataFrame(rows)

    return pd.DataFrame(rows)


In [52]:
df = photons_to_dataframe(raw_file_example, max_rows=50_000) 
df.head()


AttributeError: 'I3CompressedPhoton' object has no attribute 'zenith'